# RMN Baseline

Training an RMN on sessions 105 - 111

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
os.chdir("../../../scripts/assembly/")

In [3]:
from preprocess import *
from document import *
from subject import subject_keywords
from constant import SPEECHES, SPEAKER_MAP, HB_PATH, EMBEDDINGS, DOC_PATH, DOCUMENT

In [4]:
os.chdir("../modeling")
from token_mapping import ohe_attributes, build_tokenizer_dict, build_metadata_dict

In [5]:
DOC_SAMPLE_PATH = os.path.join(DOC_PATH, "doc-sample/")

In [6]:
docs_df = load_documents(subject_keywords.keys(), DOC_SAMPLE_PATH)

In [7]:
docs_df

,speakerid,lastname,firstname,chamber,state,gender,party,document,congress,subject
0,105111840,GINGRICH,NEWTON,H,GA,M,R,lyndon johnsons former advisor jimmy carters s...,105,alcohol
1,105110921,ASHCROFT,JOHN,S,MO,M,R,defective products placing burden responsible ...,105,alcohol
2,105114091,DODD,CHRISTOPHER,S,CT,M,D,supervision devastating longterm impact one st...,105,alcohol
3,105111431,MOYNIHAN,DANIEL,S,NY,M,D,protracted series negotiations ensued able upd...,105,alcohol
4,105116401,INOUYE,DANIEL,S,HI,M,D,days hospitalization emergency room visits vi...,105,alcohol
...,...,...,...,...,...,...,...,...,...,...
163292,111121290,SCOTT,ROBERT,H,VA,M,D,time consume first want well introducing impor...,111,trade
163293,111121290,SCOTT,ROBERT,H,VA,M,D,provisions false claims act order effectively ...,111,trade
163294,111121290,SCOTT,ROBERT,H,VA,M,D,consensus worked together bipartisan basis sen...,111,trade
163295,111121290,SCOTT,ROBERT,H,VA,M,D,yielding along colleague would like briefly di...,111,trade


In [8]:
docs_df.dtypes

speakerid    object
lastname     object
firstname    object
chamber      object
state        object
gender       object
party        object
document     object
congress     object
subject      object
dtype: object

In [32]:
feature_columns = [
 'speakerid',
 'chamber',
 'state',
 'gender',
 'party',
 'congress',
 'subject']

In [54]:
token_dict = build_tokenizer_dict(docs_df, feature_columns)

In [86]:
metadata_dict = build_metadata_dict(docs_df, feature_columns)
metadata_dict.keys()

dict_keys(['speakerid', 'chamber', 'state', 'gender', 'party', 'congress', 'subject'])

In [56]:
os.chdir("../modeling")
os.listdir(os.getcwd())

from embeddings import *
from orthoganlity_constraint import Orthoganal
from rmn import RMN

In [81]:
import pickle
embedding_file = "/home/rocassius/gen-data/tools/embbedding_index_50d"

f = open(embedding_file, "rb")
embeddings_index = pickle.load(f)
f.close()

embeddings_matrix = build_embedding_matrix(token_dict['document']['token_index'], embeddings_index)

In [73]:
# average of spane embeddings
Vst_train = embeddings_matrix[token_dict['document']['tokenized']].mean(axis=1)
Vst_train

MemoryError: Unable to allocate array with shape (1997661, 50, 50) and data type float64

In [87]:
inputs = [Vst_train] + [np.array(token_dict[col]['tokenized']) for col in metadata_dict.keys()]

In [98]:
np.random.seed(565)
rmn = RMN()
rmn.build_model(metadata_dict)
rmn.model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
speakerid.Input (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
chamber.Input (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
state.Input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
gender.Input (InputLayer)       [(None, 1)]          0                                            
____________________________________________________________________________________________

In [99]:
rmn.model.fit(x = inputs, y = Vst_train, batch_size = 1, epochs = 10)

AttributeError: 'list' object has no attribute 'shape'